## Structure and Layout

To start coding your project, we highly suggest following a clear structure in your code. You can do this by keeping the main app.py simple, with plenty callbacks to helper functions you can define in other files, to keep simplicity. Here, we will use such a structure. In this example the components connect everything together so that everything responds dynamically to tab selection by using a single callback function that connects the tab UI, data loaders, and visualisation builders. Here's how that flow works in practice:

* **User selects tabs** – The dashboard has two tab sets: graph-tabs (which analysis view to show) and data-tabs (whether to use Movie or Series data). These serve as inputs to the central callback.

* **Callback receives tab values** – The @app.callback decorator in app.py listens for changes to these tabs. It sends the selected values into the update_tab() function.

* **Data is loaded** – The load_data(tab) helper selects the correct .csv and .xlsx combo based on whether the user is exploring movies or series.

* **Builder function is selected** – Depending on the analysis tab (overview, content_creators, etc.), the callback looks up the appropriate visualisation generator (like generate_visualizations1). This lookup happens via a dictionary (VISUALIZATION_BUILDERS), making the logic declarative and easy to extend.

* **Figures are generated** – The selected builder function is run with the loaded data, returning a fixed number of Plotly figures (2 or 4 depending on the tab).

* **Figures are wrapped into layout** – These figures are passed into wrap_figures(), a helper that arranges them into a 2-column grid using dcc.Graph components.

* **Dash injects the layout** – The wrapped layout is returned as the output of the callback, replacing the contents of <div id='tabs-content'> in the app layout.

This pattern is powerful because it decouples content generation from layout logic, and avoids repetitive if-else blocks. Adding a new tab requires only adding a new builder function and registering it in a dictionary—no need to rewrite the callback itself.

We recommend checking out the full app.py code and the .py files in /src to get a good feel for it yourself. Below, we'll go more in depth on this code, to understand how it works exactly, so that you can implement some of its elements in your own project.

---


### Setup
To start, we'll have to do some simple imports. Make sure you import all relevant features that you want to use from dash, and the helper functions you'll use for the visualizations and data extraction:

In [4]:
from pathlib import Path #i like using this path library to have a set path saved, kind of as a variable, that we can then reuse everywhere, but you don't have to use this.

import dash_bootstrap_components as dbc #see explanation of bootstrap below
import pandas as pd #for database management (dataframes)
from dash import Dash, dcc, html, Input, Output

from src.const import get_constants #helper function to get constants from our datasets
from src import dash1, dash2, dash3, dash4 #helper functions to create the visualizations based on selected tabs.

**What's Bootstrap?**

Bootstrap is a CSS framework that helps you build responsive and well-structured UIs using a 12-column grid and pre-styled components like cards, buttons, and rows. In our case, we use Dash Bootstrap Components (dbc) to bring Bootstrap's layout and styling into the Dash ecosystem. 

We don't actually use .css files here, as bootstrap can handle it on its own, without that. Though you are free to use a custom .css file if that aligns with your dashboard, there is a seperate .css notebook in the Github that will go into this shortly.

In this IMDb dashboard example: 
* Bootstrap provides the grid system (dbc.Container, dbc.Row, dbc.Col) for clean layout and automatic responsiveness.
* It also gives us themeable UI elements like dbc.Card, which we use for KPI tiles.
* Dash handles the interactivity (via Python + callbacks), while Bootstrap ensures the app looks good on all screen sizes without writing custom CSS.

Think of Bootstrap as the design scaffolding, and Dash as the brain.

---

Then, before we define the layout and logic, we define a few global constants and mappings. These support dynamic tab logic, dropdown filtering, card rendering, and consistent styling across the app.

We load our datasets, in this case, the movie and series datasets (both .csv and .xlsx) once at startup using Path(...) / filename, which is cleaner and OS-agnostic.

In [ ]:
DATA_DIR = Path("./data") #whatever the path to your data is

MOVIES = pd.read_csv(DATA_DIR / "movie_after_cleaning.csv")
MOVIES_SPLITS = pd.read_excel(DATA_DIR / "splits_movie.xlsx", sheet_name=None)
SERIES = pd.read_csv(DATA_DIR / "series_after_cleaning.csv")
SERIES_SPLITS = pd.read_excel(DATA_DIR / "splits_series.xlsx", sheet_name=None)

Instead of using if tab == ..., we use dictionaries for clarity and extensibility:

In [ ]:
DATA_BY_TAB = {
    "movie": (MOVIES, MOVIES_SPLITS),
    "series": (SERIES, SERIES_SPLITS),
}

So inside the callback we can just do something like:
"data, splits = DATA_BY_TAB[data_tab]"

However, if you are not familiar with using a dictionary (or just don't want to :P), you can use a simple if–elif structure like this:
```
if data_tab == "movie":
    data, splits = MOVIES, MOVIES_SPLITS
elif data_tab == "series":
    data, splits = SERIES, SERIES_SPLITS
```